In [88]:
import os
import pandas as pd

directory = '../PruningProfiler/output/2024-04-17_Vaisakh'
dataframes = []

def process_model_perfstats(modelname, filepath):
    # columns = ['metric', 'values']
    df = pd.read_csv(filepath+'/system_stats.csv')
    df.loc[len(df.index)] = ['modelname', modelname]
    df = df.T
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    # display(df)
    return df

combined_df = pd.DataFrame()
first_time = True
for filename in os.listdir(directory):
    for iterconfig_filename in os.listdir(os.path.join(directory, filename)):
        filepath = os.path.join(directory, filename, iterconfig_filename)
        ## We can use this for naming file or adding columns
        modelname, prune_conf, prune_ratio = iterconfig_filename.split('-')
        # Get the "system_stats.csv" file and process it
        newdf = process_model_perfstats(modelname, filepath)
        if first_time:
            combined_df = newdf
            first_time = False
        else:
            combined_df = pd.concat([combined_df, newdf])
print (len(combined_df.columns))
print (combined_df.columns)
newcols = ["modelname", 
           "prune_config", "prune_ratio",
            "acc", 
            "chipset_name", "arch", "l1_size", "l2_size", 
            "cpuloadavg_1min", "cpuloadavg_5min", "cpu_psi_total", "io_psi_total",
            "vm_percent", "swap_percent"]
combined_df = combined_df[newcols]
combined_df = combined_df.sort_values(by = ['modelname', 'prune_config', 'prune_ratio'], ascending=[True, True, True])
display(combined_df)


17
Index(['vm_percent', 'swap_percent', 'cpuloadavg_1min', 'cpuloadavg_5min',
       'cpu_0', 'cpu_1', 'cpu_psi_total', 'mem_psi_total', 'io_psi_total',
       'acc', 'prune_config', 'prune_ratio', 'chipset_name', 'arch', 'l1_size',
       'l2_size', 'modelname'],
      dtype='object', name='Unnamed: 0')


Unnamed: 0,modelname,prune_config,prune_ratio,acc,chipset_name,arch,l1_size,l2_size,cpuloadavg_1min,cpuloadavg_5min,cpu_psi_total,io_psi_total,vm_percent,swap_percent
0,alexnet,l1_unstructured,0.0,59.00000333786011,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.51465395480226,1.5065324858757063,334916580.9237288,192097739.6355932,2.0788135593220343,0.0
0,alexnet,l1_unstructured,0.1,59.00000333786011,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.544859514687101,1.5257556406981696,335279879.29501915,192113140.52490422,2.0310344827586206,0.0
0,alexnet,l1_unstructured,0.2,59.400004148483276,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.575,1.5434670781893005,335654008.1111111,192142592.25555557,2.0288888888888885,0.0
0,alexnet,l1_unstructured,0.30000000000000004,58.799999952316284,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.5706882255389718,1.5691853233830841,336024920.2835821,192167027.88059703,2.0417910447761196,0.0
0,alexnet,l1_unstructured,0.4,58.60000252723694,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.524463383838384,1.509680134680135,336424135.8068182,192180073.13257575,2.0571969696969696,0.0
0,alexnet,l1_unstructured,0.5,58.40000510215759,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.560341240116521,1.5459841864336248,336830634.0486891,192192602.6779026,2.0232209737827715,0.0
0,alexnet,l1_unstructured,0.6000000000000001,56.80000185966492,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.5328092243186584,1.5255241090146752,337234747.84528303,192205095.58490565,2.019622641509434,0.0
0,alexnet,l1_unstructured,0.7000000000000001,56.40000104904175,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.5908239700374531,1.5687161880982103,337651002.3820225,192237584.4906367,2.0205992509363297,0.0
0,alexnet,ln_structured,0.0,59.00000333786011,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.5760173754000917,1.55778463648834,338112351.2427983,192255946.1851852,2.0732510288065846,0.0
0,alexnet,ln_structured,0.1,52.000004053115845,Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz,x86_64,1.1 MiB,36 MiB,1.5436347278452542,1.5378430049482683,338506785.11336035,192269605.3117409,2.0894736842105264,0.0
